### **BigBasket Product's Query Engine Notebook 3**
Note book is aimed to integrating vector dd qdrant llm model 

**It Contains**

**1. Important imports**
1. qdrant space 
2. bert model 
3. st encoder model 

**2. searching closest context from vector db and returning respose from context**

In [1]:
!pip install -q qdrant-client
!pip install -q sentence-transformers


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.1 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.1 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires protobuf<4.0.0dev,>=3.12.0, but you have protobuf 4.25.1 which is incompatible.
google-cloud-bigtable 1.7.3 requires protobuf<4.0.0dev, but you have protobuf 4.25.1 which is incompatible.
google-cloud-datastore 

In [2]:
import pickle
from sentence_transformers import SentenceTransformer


In [3]:
# VECTOR_SPACE_PATH = "/kaggle/input/chaabi/space.pkl"
BERT_MODEL_PATH = "/kaggle/input/chaabi/bert-question-answering.pkl"
ENC_PATH  = "/kaggle/input/chaabi/encodermodel.pkl"
QDRANT_PATH  = "/kaggle/input/chaabi/qdrant_space_client.pkl"


# with open(VECTOR_SPACE_PATH, 'rb') as file:
#     vs = pickle.load(file)
# bert llm
with open(BERT_MODEL_PATH, 'rb') as file:
    bert = pickle.load(file)


    
    
    

In [4]:
# qdrant clinet to interact qdrant space containing all vectors 

with open(QDRANT_PATH, 'rb') as file:
    qdrant_client = pickle.load(file)

In [5]:
# sentence transformer encoder 

with open(ENC_PATH, 'rb') as file:
    st_encoder = pickle.load(file)
    


In [6]:
collection_name = "qdrant-space"
def find_close_contexts(question: str, top_k: int) -> list[str]:
    """
    will return contexts contexts close to query 

    Args:
        question (str): What do we want to know?
        top_k (int): top k results will be added 

    Returns:
        context (List[str]):
    """
    try:
        encoded_query = st_encoder.encode(question).tolist() 
        result = qdrant_client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer

        context = [
            [context.payload["product"], context.payload["story"]] for context in result
        ] 
        return context
    except Exception as e:
        print({e})

In [7]:
def tell_me(question: str, context: list[str]):
    """
    Extract the answer from the context for a given question

    Args:
        question (str): _description_
        context (list[str]): _description_
    """
    results = []
    for c in context:
        answer = bert(question=question, context=c[1] )
        answer["product"] = c[0]
        results.append(answer)
        print()

    sorted_result = sorted(results, key=lambda x: x["score"], reverse=True)
    for i in range(len(sorted_result)):
        _out = sorted_result[i]["answer"] 
        _prod = sorted_result[i]["product"]
        _sco = sorted_result[i]["score"]
#         print(f"{i+1}", end=" ")
        print(f"QUERY INPUT: {question}")
        print(f"OUTPUT: {_out} \nPREDICTION SCORE {_sco}\n\nReferred Product: {_prod}\n\n")
        return question,_out,_sco,_prod




In [8]:
queries = []
queries.append("suggest me one product for cleaning vegetables")
queries.append("what is the rating of product Vegetable & Fruit Wash with 100% Natural Action")
queries.append("what is most loved beauty product")
queries.append("price of dove soap")
queries.append("what is most loved beauty product")
queries.append("suggest one Tea Product")



In [9]:
with open("result.txt","w") as result:
    for q in queries:
        
        c = find_close_contexts(q, top_k=1)
        _ques,_out,_sco,_prod = tell_me(q,c)
        result.write(f"QUERY INPUT: {_ques}\nOUTPUT: {_out} \nPREDICTION SCORE {_sco}\n\nReferred Product: {_prod}\n\n")
    

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


QUERY INPUT: suggest me one product for cleaning vegetables
OUTPUT: Vegetable & Fruit Wash 
PREDICTION SCORE 0.003299231640994549

Referred Product: Vegetable & Fruit Wash with 100% Natural Action




Batches:   0%|          | 0/1 [00:00<?, ?it/s]


QUERY INPUT: what is the rating of product Vegetable & Fruit Wash with 100% Natural Action
OUTPUT: 4.3 
PREDICTION SCORE 0.9139595627784729

Referred Product: Vegetable & Fruit Wash with 100% Natural Action




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...


QUERY INPUT: price of dove soap
OUTPUT: 699.0 
PREDICTION SCORE 0.4523414671421051

Referred Product: Lavender Soap Spa Set




Batches:   0%|          | 0/1 [00:00<?, ?it/s]


QUERY INPUT: what is most loved beauty product
OUTPUT: Brightening Beauty Pack 
PREDICTION SCORE 0.11472972482442856

Referred Product: Brightening Beauty Pack




Batches:   0%|          | 0/1 [00:00<?, ?it/s]


QUERY INPUT: suggest one Tea Product
OUTPUT: Go ahead and give this one a shot without a second thought 
PREDICTION SCORE 0.12469898164272308

Referred Product: Darjeeling Black Tea


